Concept

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [6]:
from sklearn.tree import DecisionTreeRegressor

In [8]:
# cd ../SL_ANN

In [9]:
df = pd.read_csv('data/kc_house_data.csv')

In [31]:
df = df.drop('id',axis=1)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].apply(lambda date:date.month)
df['year'] = df['date'].apply(lambda date:date.year)
df = df.drop('date', axis=1)
df = df.drop('zipcode',axis=1)

X = df.drop('price',axis=1)
y = df['price'].values


KeyError: "['id'] not found in axis"

In [11]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [13]:
y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [17]:
y_pred

array([218915.92922165, 550136.45513139, 429244.37686992, ...,
       429244.37686992, 429244.37686992, 429244.37686992])

In [18]:
y

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

Gradient Boosting Algorithm

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

In [25]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=120)

In [26]:
gbrt.predict(X)

array([200390.68493472, 434255.41045857, 321905.53200136, ...,
       378389.29269548, 432868.2114082 , 350482.00296813])

In [27]:
y

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [28]:
gbrt.score(X,y)

0.912936528944298

Find optimal number of trees:

In [29]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X,y)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=1000)
gbrt.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=1000)

In [38]:
errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
best_n_estimators = np.argmin(errors)+1


In [39]:
best_n_estimators

995

Implement Early Stopping

In [43]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up=0
    else:
        error_going_up+= 1
        if error_going_up == 5:
            print(n_estimators)
            break #early stopping
        


110


XGBoost

In [44]:
import xgboost

c:\Users\antho\anaconda3\envs\mytfenv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [45]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)

c:\Users\antho\anaconda3\envs\mytfenv\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [46]:
y_pred = xgb_reg.predict(X_val)

In [51]:
y_pred

array([653191.25, 338639.66, 264311.03, ..., 543789.25, 888178.5 ,
       944268.4 ], dtype=float32)

In [52]:
y_val

array([637000., 285750., 270000., ..., 590000., 740000., 905000.])